## PACKAGES

In [98]:
# packages
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime as dt

## DATA

In [113]:
df1 = pd.read_csv('../goodtrainbadtrain/data/select_2020.csv',encoding='iso-8859-2',sep=';')
df2 = pd.read_csv('../goodtrainbadtrain/data/select_2021.csv',encoding='iso-8859-2',sep=';')
df3 = pd.read_csv('../goodtrainbadtrain/data/select_2022.csv',encoding='iso-8859-2',sep=';')

df = pd.concat([df1, df2, df3])


## FILTER

In [114]:
#filter for München and Köln
df = df.query("bhf in ('München Hbf', 'Köln Hbf', 'Köln Messe/Deutz Gl.11-12')")

## PROCESS AND CLEAN

In [115]:
#1) cleaning of train names 
#some letters of 'zugnr' are not capitalized
df['zugnr'] = df['zugnr'].str.upper()

#2)process of 9999 in arrTime and depTime: add new column with info 
df['start_or_endpoint'] = 'nan'
df.loc[df['arrTime'] == 9999, 'start_or_endpoint'] = 'start'
df.loc[df['depTime'] == 9999, 'start_or_endpoint'] = 'end'
#overwrite 9999 with respective arr/dep time of same observation (in new clean columns)
df['arrTime_clean'] = np.where(df['arrTime'] == 9999, df['depTime'], df['arrTime'])
df['depTime_clean'] = np.where(df['depTime'] == 9999, df['arrTime'], df['depTime'])

#3)some times need to be filled up with 0's. eg. '5' -> 00:05
df['arrTime_clean'] = df['arrTime_clean'].astype(str)
df['arrTime_clean'] = df['arrTime_clean'].map(lambda a: a.zfill(4))
df['depTime_clean'] = df['depTime_clean'].astype(str)
df['depTime_clean'] = df['depTime_clean'].map(lambda a: a.zfill(4))

# FEATURE ENGENEERING

In [116]:
#1)add city feature (merged Köln Hbf/ Messe Deutz)
df['city'] = df.bhf
df['city'] = df['city'].str.replace('Köln Messe/Deutz Gl.11-12','Köln')
df['city'] = df['city'].str.replace('Köln Hbf','Köln')
df['city'] = df['city'].str.replace('München Hbf','München')

#2) add date column
df['date'] = df['datum'] + ' ' + df['arrTime_clean']
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H%M')
#3) add month of the year 
df['weekday'] = df['date'].dt.day_name()
#4) add month of the year 
df['month'] = df['date'].dt.month_name()

#time of the day
df['time_of_day'] = pd.cut(pd.to_datetime(df.date).dt.hour,
       bins=[0, 6, 12, 18, 24],
       labels=['night', 'morning', 'afternoon', 'evening'],
       right=False,
       include_lowest=True)

#todo
#3) add public holiday
#4) add covid lockdown?

/var/folders/0v/92j280qj2j971syqfbthtx0r0000gn/T/ipykernel_12429/3253485097.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['city'] = df['city'].str.replace('Köln Messe/Deutz Gl.11-12','Köln')


### ADD DIRECTON INFO (SO FAR OF SIMPLE JOURNEYS)

In [117]:
journeys = pd.read_csv('../goodtrainbadtrain/data/journeyindex.csv')

#processing
journeys = journeys[journeys.leg1_train.notna()] #delete duplicated trips that once go to köln hbf and once to deutz
journeys = journeys[journeys.leg2_train.notna()] 
journeys = journeys[journeys.leg3_train.notna()] 

journeys = journeys.drop(['Unnamed: 0'],axis=1) 
journeys['key_ID'] = list(range(journeys.shape[0]))

#rename columns (necessary for wide_to_long function)
journeys.columns = [ 'date', 'weekday', 'month', 'journey_origin',
       'journey_destination', 'journey_start', 'journey_end',
       'journey_duration', 'journey_numberlegs', 
       'train_leg1', 'origin_leg1','destination_leg1', 'start_leg1', 'end_leg1', 'duration_leg1',
       'train_leg2', 'origin_leg2', 'destination_leg2', 'start_leg2','end_leg2', 'duration_leg2', 
       'train_leg3', 'origin_leg3','destination_leg3', 'start_leg3', 'end_leg3', 'duration_leg3', 
       'key_ID']

journeys_long = pd.wide_to_long(df = journeys,
                                stubnames=['train', 'origin','destination', 'start', 'end', 'duration'],
                                i=['key_ID'],
                                j='leg',
                                sep = '_',
                                suffix='.+').reset_index()

#delete empty legs
journeys_long = journeys_long[journeys_long.train != '-1']

#reorder columns
journeys_long = journeys_long[['key_ID', 
 'journey_origin', 'journey_destination','journey_start','journey_end','journey_duration', 'journey_numberlegs',
 'leg', 'train', 'origin', 'destination', 'start', 'end','duration',
 'date', 'month','weekday']]


def date_transformation(df, columns):
    for col in columns:
        df[col] = df[col].str.split('+', expand=True)[[0]]
        df[col] = pd.to_datetime(df[col])
    return df


journeys_long = date_transformation(journeys_long, ['journey_start', 'journey_end']) #, 'start', 'end'])
journeys_long['month'] = journeys_long['journey_start'].dt.month_name()
journeys_long['weekday'] = journeys_long['journey_start'].dt.day_name()


simple_journeys = journeys[journeys.journey_numberlegs == 1]
simple_journeys_tomunich = simple_journeys[simple_journeys.journey_destination == "München Hbf"].train_leg1.unique()
simple_journeys_tocologne = simple_journeys[(simple_journeys.journey_destination == "Köln Hbf") | (simple_journeys.journey_destination == "Köln Messe/Deutz Gl.11-12")].train_leg1.unique()


df['ends_cologne'] = df['zugnr'].isin(simple_journeys_tocologne.tolist())
df['ends_munich'] = df['zugnr'].isin(simple_journeys_tomunich.tolist())


cologne_mask = (df.ends_cologne) & (df.city == 'Köln')
munich_mask = (df.ends_munich) & (df.city == 'München')
df = df[cologne_mask | munich_mask]

München Hbf                         451
Frankfurt(Main) Flughafen Fernbf    225
Köln Messe/Deutz Gl.11-12           202
Mannheim Hbf                        180
Köln Hbf                            165
Stuttgart Hbf                        81
Würzburg Hbf                         41
Frankfurt(Main) Hbf                  39
Nürnberg Hbf                         10
Name: origin, dtype: int64

# ADD WEATHER

In [118]:
#5) add weather variables
df['sharp_date'] = df['date'].dt.round('H')
df['sharp_date']  = df.sharp_date.astype('str')

cgn_data = pd.read_csv('../goodtrainbadtrain/data/koln.csv')
cgn_data['sharp_date']  = cgn_data.time.astype('str')

muc_data = pd.read_csv('../goodtrainbadtrain/data/munchen.csv')
muc_data['sharp_date'] = muc_data.time.astype('str')

weather = {"Köln": cgn_data, "München": muc_data}


total_df =  pd.DataFrame()
for station, w_df in weather.items():
    station_df = pd.merge(df[df['city'] == station], weather[station], how='left', left_on='sharp_date', right_on='sharp_date') #, right_index=True)
    total_df = pd.concat([total_df, station_df])

df = total_df.drop(columns=['dwpt', 'rhum', 'wdir', 'pres', 'tsun'])
df['snow'] = df['snow'].replace(np.nan, 0)

# Load coco file
coco = pd.read_csv('../goodtrainbadtrain/data/weather_coco.csv', sep=';')
coco.set_index('Code', inplace=True)
coco = coco.to_dict()['Weather Condition']

# Define new classification for coco
new_classes = {
    'good': [1, 2],
    'medium': [3, 4, 7, 14],
    'bad': [5, 8, 10, 12, 15, 17, 19, 21, 23, 24, 25],
    'extreme': [6, 9, 11, 13, 16, 18, 20, 22, 26, 27]
}

# Apply new classification for coco
reclass = {}
for k, values in new_classes.items():
    for v in values:
        for c in range(1, 28):
            if v == c:
                reclass[v] = k

reclass = dict(sorted(reclass.items()))
df['coco'] = df['coco'].map(reclass)

# TARGET PROCESSING

In [119]:
#1) into several categories
# adelay-> into categories: no delay, small delay, medium delay, (big delay/cancellation)
max = df.adelay.max()
bins = [-2, -0.1, 0, 5, 30, max]
group_names = ['large delay/cancelled','on time','small delay', 'medium delay', 'large delay/cancelled']
df['target'] = pd.cut(df['adelay'], bins, labels=group_names, ordered=False)

#value counts of target:
#on time                  67336
#small delay              28649
#medium delay             23617
#large delay/cancelled    10677

#2) binary target (on time - or not)
df['target_binary'] =  (df['adelay'] == 0)*1

#3) numeric target (cancelled and extreme values = 120 Min)
df['target_numeric'] = df['adelay']
df['target_numeric'] = np.where(df['target_numeric'] == -1, 120, df['target_numeric'])
df['target_numeric'] = np.where(df['target_numeric'] > 120, 120, df['target_numeric'])
#sns.boxplot(df['target_numeric'])

# CORRELATION / TARGET VISUALIZATION

In [120]:
#sns.barplot(x= df.weekday, y=df.target_numeric)

In [121]:
#sns.barplot(x= df.month, y=df.target_numeric)

# SAVE DATA

In [124]:
df.shape

(40079, 28)

In [125]:
df.to_csv('../goodtrainbadtrain/data/data_for_model_2.csv')